# Topic Modeling

## Introduction

I'll be doing topic modeling for the newspaper headlines, though I don't expect to find much difference in between the papers, maybe a topic or two will change.

For that I'll be using **Latent Dirichlet Allocation (LDA)**, which is a topic modeling technique, with the *Gensim* package and *Spacy*.

In regards to inputs and outputs, I'll be working with the **Document Term Matrix (DTM)** as en inpus, and the outputs will be list with topics for each newspaper.

In [1]:
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import spacy

from dotenv import load_dotenv
from itertools import product
from gensim import matutils, models
from scipy import sparse

In [2]:
load_dotenv()

BASE_DIR = os.environ.get("BASE_DIR")
BEARER_TOKEN = os.environ.get("BEARER_TOKEN")

In [3]:
pd.set_option("display.max_colwidth", 300)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 50)
pd.set_option("display.precision", 2)
pd.set_option("display.float_format",  "{:,.3f}".format)

pio.templates.default = "plotly_white"
pio.kaleido.scope.default_scale = 2

gruvbox_colors = ["#fabd2f", "#b8bb26", "#458588", "#fe8019", "#b16286", "#fb4943", "#689d6a", "#d79921", "#98971a", "#83a598", "#d65d0e", "#d3869b", "#cc241d", "#8ec07c", "#b57614", "#79740e", "#076678", "#af3a03", "#8f3f71", "#9d0006", "#4d7b58", "#fbf1c7", "#928374", "#282828"]

In [4]:
TIME_STAMPS = [(2022, 35), (2022, 40), (2022, 45), (2022, 50), (2023, 3)]

In [5]:
dtm = pd.read_pickle(f"{BASE_DIR}/data/processed/dtm-{TIME_STAMPS[0]}-{TIME_STAMPS[-1]}.pkl")
stats_data = pd.read_feather(f"{BASE_DIR}/data/processed/stats_data-{TIME_STAMPS[0]}-{TIME_STAMPS[-1]}.feather")
data_dtm = pd.read_pickle(f"{BASE_DIR}/data/processed/data-dtm-{TIME_STAMPS[0]}-{TIME_STAMPS[-1]}.pkl")

In [6]:
stats_data["year"] = stats_data["created_at"].dt.isocalendar().year
stats_data["week"] = stats_data["created_at"].dt.isocalendar().week

data_dtm["year"] = data_dtm["created_at"].dt.isocalendar().year
data_dtm["week"] = data_dtm["created_at"].dt.isocalendar().week

In [7]:
data_dtm["year_week"] = data_dtm["year"].astype("str") + "w" + data_dtm["week"].astype("str")

In [8]:
newspapers = data_dtm["newspaper"].unique()
year_week = data_dtm["year_week"].unique()

In [9]:
data_dtm.head()

,id,created_at,newspaper,corpus,doc,token,lemma,year,week,year_week
0,1564039479391838209,2022-08-28 23:57:24+00:00,elcomercio_peru,venezuela colombia retoman relaciones diplomáticas rotas hace tres años,"(venezuela, colombia, retoman, relaciones, diplomáticas, rotas, hace, tres, años)","[venezuela, colombia, retoman, relaciones, diplomáticas, rotas, años]","[venezuela, colombia, retomar, relación, diplomático, roto, año]",2022,34,2022w34
1,1564032331706470401,2022-08-28 23:29:00+00:00,elcomercio_peru,amlo afirma que familias ya aceptaron plan de rescate de mineros,"(amlo, afirma, que, familias, ya, aceptaron, plan, de, rescate, de, mineros)","[amlo, afirma, familias, aceptaron, plan, rescate, mineros]","[amlo, afirmar, familia, aceptar, plan, rescate, minero]",2022,34,2022w34
2,1564028601053347843,2022-08-28 23:14:11+00:00,elcomercio_peru,zelensky los ocupantes rusos sentirán las consecuencias de futuras acciones,"(zelensky, los, ocupantes, rusos, sentirán, las, consecuencias, de, futuras, acciones)","[zelensky, ocupantes, rusos, sentirán, consecuencias, futuras, acciones]","[zelensky, ocupante, ruso, sentir, consecuencia, futuro, acción]",2022,34,2022w34
3,1564023766937731073,2022-08-28 22:54:58+00:00,elcomercio_peru,autoridades confirman transmisión comunitaria de viruela del mono en panamá,"(autoridades, confirman, transmisión, comunitaria, de, viruela, del, mono, en, panamá)","[autoridades, confirman, transmisión, comunitaria, viruela, mono, panamá]","[autoridad, confirmar, transmisión, comunitario, viruela, mono, panamá]",2022,34,2022w34
4,1564017585561141248,2022-08-28 22:30:25+00:00,elcomercio_peru,las imágenes de los enfrentamientos entre seguidores de cristina kirchner la policía en argentina,"(las, imágenes, de, los, enfrentamientos, entre, seguidores, de, cristina, kirchner, la, policía, en, argentina)","[imágenes, enfrentamientos, seguidores, cristina, kirchner, policía, argentina]","[imagen, enfrentamiento, seguidor, cristina, kirchner, policía, argentina]",2022,34,2022w34


In [10]:
data_dtm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34851 entries, 0 to 34923
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   id          34851 non-null  object             
 1   created_at  34851 non-null  datetime64[ns, UTC]
 2   newspaper   34851 non-null  object             
 3   corpus      34851 non-null  object             
 4   doc         34851 non-null  object             
 5   token       34851 non-null  object             
 6   lemma       34851 non-null  object             
 7   year        34851 non-null  UInt32             
 8   week        34851 non-null  UInt32             
 9   year_week   34851 non-null  object             
dtypes: UInt32(2), datetime64[ns, UTC](1), object(7)
memory usage: 2.7+ MB


After reading **Gensim's documentation** I noticed that to get better results I need to lemmatize the words so I'll be redoing the DTM using **Spacy**.

In [11]:
dtm.head()

lemma,aa,aaaaatención,aaaatención,aactor,aafp,aaguinagar,aahh,aap,aar,abad,abajo,abancay,abanderar,abandona,abandonado,abandonar,abandono,abandón,abantir,abanto,abarcar,abarrot,abarrotar,abastecer,abastecer él,...,ünsal,​chocolate,​mantequilla,⃣,→,↓,─,⦿,𝗔𝘂𝗱𝗶𝘁𝗼𝗿𝗶𝗼,𝗖𝗮𝘁𝗮́𝗹𝗼𝗴𝗼,𝗗𝗲𝗹,𝗘𝗱𝗶𝘁𝗼𝗿𝗮,𝗣𝗮𝘁𝗿𝗶𝗰𝗶𝗮,𝗣𝗲𝗿𝘂́,𝗦𝗮́𝗯𝗮𝗱𝗼,𝗨́𝗻𝗲𝘁𝗲,𝗩𝗮𝗹𝗹𝗲,𝗱𝗲,𝗱𝗲𝗹,𝗲𝗱𝗶𝘁𝗼𝗿𝗶𝗮𝗹,𝗵𝗿𝘀,𝗹𝗮,𝗻𝗼𝘃𝗶𝗲𝗺𝗯𝗿𝗲,𝗽𝗿𝗲𝘀𝗲𝗻𝘁𝗮𝗰𝗶𝗼́𝗻,󠁧󠁢󠁥󠁮󠁧󠁿
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1558966707611385861,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1558966968039997441,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1558967193043361792,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1558967616777109510,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1558968396674473985,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [52]:
newspapers_dtm = {}
newspapers_models = {}

for newspaper in newspapers:

    dtm_per_week = []

    for period in year_week:
        ids = data_dtm.loc[(data_dtm["newspaper"] == newspaper) & (data_dtm["year_week"] == period), "id"].tolist()

        week_tweets = dtm.T[ids]
        week_dtm = pd.DataFrame(week_tweets.T.sum())
        week_dtm.rename({0: period}, inplace=True, axis=1)

        dtm_per_week.append(week_dtm)

    newspaper_tweets = pd.concat(dtm_per_week, axis=1)

    newspapers_dtm[newspaper] = newspaper_tweets
    
    sparse_dtm = sparse.csc_matrix(newspaper_tweets)
    tweet_corpus = matutils.Sparse2Corpus(sparse_dtm)

    id2word = {}

    for index, word in enumerate(newspaper_tweets.T.columns):
        id2word[index] = word

    newspapers_models[newspaper] = models.LdaModel(corpus=tweet_corpus, id2word=id2word, num_topics=3, passes=80)

In [53]:
newspaper_lda = pd.DataFrame.from_dict(newspapers_models, orient="index")

In [54]:
newspaper_lda.reset_index(inplace=True)
newspaper_lda.rename({0: "lda_model"}, axis=1, inplace=True)

In [55]:
newspaper_lda.head()

,index,lda_model
0,elcomercio_peru,"LdaModel<num_terms=24473, num_topics=3, decay=0.5, chunksize=2000>"
1,larepublica_pe,"LdaModel<num_terms=24473, num_topics=3, decay=0.5, chunksize=2000>"
2,peru21noticias,"LdaModel<num_terms=24473, num_topics=3, decay=0.5, chunksize=2000>"
3,tromepe,"LdaModel<num_terms=24473, num_topics=3, decay=0.5, chunksize=2000>"
4,Gestionpe,"LdaModel<num_terms=24473, num_topics=3, decay=0.5, chunksize=2000>"


In [56]:
newspaper_lda["topics"] = newspaper_lda["lda_model"].apply(lambda lda: lda.print_topics())

In [57]:
newspaper_lda["topic_1"] = newspaper_lda["topics"].apply(lambda x: x[0])
newspaper_lda["topic_2"] = newspaper_lda["topics"].apply(lambda x: x[1])
newspaper_lda["topic_3"] = newspaper_lda["topics"].apply(lambda x: x[2])
# newspaper_lda["topic_4"] = newspaper_lda["topics"].apply(lambda x: x[3])
# newspaper_lda["topic_5"] = newspaper_lda["topics"].apply(lambda x: x[4])
# newspaper_lda["topic_6"] = newspaper_lda["topics"].apply(lambda x: x[5])
# newspaper_lda["topic_7"] = newspaper_lda["topics"].apply(lambda x: x[6])
# newspaper_lda["topic_8"] = newspaper_lda["topics"].apply(lambda x: x[7])
# newspaper_lda["topic_9"] = newspaper_lda["topics"].apply(lambda x: x[8])
# newspaper_lda["topic_10"] = newspaper_lda["topics"].apply(lambda x: x[9])

In [58]:
newspaper_lda.head()

,index,lda_model,topics,topic_1,topic_2,topic_3
0,elcomercio_peru,"LdaModel<num_terms=24473, num_topics=3, decay=0.5, chunksize=2000>","[(0, 0.000*""noviembre"" + 0.000*""perú"" + 0.000*""lima"" + 0.000*""millón"" + 0.000*""año"" + 0.000*""brasil"" + 0.000*""covid"" + 0.000*""eeuu"" + 0.000*""ucrania"" + 0.000*""cambio""), (1, 0.007*""perú"" + 0.004*""año"" + 0.004*""lima"" + 0.003*""covid"" + 0.003*""precio"" + 0.003*""millón"" + 0.003*""eeuu"" + 0.003*""agosto""...","(0, 0.000*""noviembre"" + 0.000*""perú"" + 0.000*""lima"" + 0.000*""millón"" + 0.000*""año"" + 0.000*""brasil"" + 0.000*""covid"" + 0.000*""eeuu"" + 0.000*""ucrania"" + 0.000*""cambio"")","(1, 0.007*""perú"" + 0.004*""año"" + 0.004*""lima"" + 0.003*""covid"" + 0.003*""precio"" + 0.003*""millón"" + 0.003*""eeuu"" + 0.003*""agosto"" + 0.003*""ucrania"" + 0.003*""gobierno"")","(2, 0.000*""perú"" + 0.000*""año"" + 0.000*""eeuu"" + 0.000*""millón"" + 0.000*""lima"" + 0.000*""méxico"" + 0.000*""unidos"" + 0.000*""rusia"" + 0.000*""ucrania"" + 0.000*""precio"")"
1,larepublica_pe,"LdaModel<num_terms=24473, num_topics=3, decay=0.5, chunksize=2000>","[(0, 0.016*""politicalr"" + 0.015*""lrdeport"" + 0.011*""video"" + 0.005*""perú"" + 0.004*""lima"" + 0.004*""castillo"" + 0.003*""año"" + 0.003*""envivo"" + 0.003*""elección"" + 0.003*""conocer""), (1, 0.017*""politicalr"" + 0.015*""lrdeport"" + 0.008*""video"" + 0.007*""castillo"" + 0.006*""perú"" + 0.005*""pedro"" + 0.005*""e...","(0, 0.016*""politicalr"" + 0.015*""lrdeport"" + 0.011*""video"" + 0.005*""perú"" + 0.004*""lima"" + 0.004*""castillo"" + 0.003*""año"" + 0.003*""envivo"" + 0.003*""elección"" + 0.003*""conocer"")","(1, 0.017*""politicalr"" + 0.015*""lrdeport"" + 0.008*""video"" + 0.007*""castillo"" + 0.006*""perú"" + 0.005*""pedro"" + 0.005*""envivo"" + 0.005*""boluarte"" + 0.005*""congreso"" + 0.004*""loúltimo"")","(2, 0.000*""politicalr"" + 0.000*""lrdeport"" + 0.000*""video"" + 0.000*""castillo"" + 0.000*""perú"" + 0.000*""loúltimo"" + 0.000*""envivo"" + 0.000*""pedro"" + 0.000*""año"" + 0.000*""lima"")"
2,peru21noticias,"LdaModel<num_terms=24473, num_topics=3, decay=0.5, chunksize=2000>","[(0, 0.000*""perú"" + 0.000*""castillo"" + 0.000*""lima"" + 0.000*""pedro"" + 0.000*""año"" + 0.000*""elección"" + 0.000*""pedir"" + 0.000*""conocer"" + 0.000*""país"" + 0.000*""candidato""), (1, 0.005*""castillo"" + 0.005*""perú"" + 0.004*""lima"" + 0.004*""pedro"" + 0.003*""año"" + 0.003*""elección"" + 0.003*""conocer"" + 0.00...","(0, 0.000*""perú"" + 0.000*""castillo"" + 0.000*""lima"" + 0.000*""pedro"" + 0.000*""año"" + 0.000*""elección"" + 0.000*""pedir"" + 0.000*""conocer"" + 0.000*""país"" + 0.000*""candidato"")","(1, 0.005*""castillo"" + 0.005*""perú"" + 0.004*""lima"" + 0.004*""pedro"" + 0.003*""año"" + 0.003*""elección"" + 0.003*""conocer"" + 0.002*""peruano"" + 0.002*""presentar"" + 0.002*""presidente"")","(2, 0.008*""castillo"" + 0.007*""pedro"" + 0.005*""perú"" + 0.004*""año"" + 0.004*""lima"" + 0.004*""peruano"" + 0.003*""congreso"" + 0.003*""conocer"" + 0.003*""boluarte"" + 0.003*""cortita"")"
3,tromepe,"LdaModel<num_terms=24473, num_topics=3, decay=0.5, chunksize=2000>","[(0, 0.004*""miss"" + 0.004*""universo"" + 0.002*""perú"" + 0.002*""canción"" + 0.002*""piqué"" + 0.002*""rovegno"" + 0.002*""leer"" + 0.002*""candidato"" + 0.002*""año"" + 0.002*""boda""), (1, 0.000*""vs"" + 0.000*""castillo"" + 0.000*""lima"" + 0.000*""perú"" + 0.000*""pedro"" + 0.000*""año"" + 0.000*""peruano"" + 0.000*""alian...","(0, 0.004*""miss"" + 0.004*""universo"" + 0.002*""perú"" + 0.002*""canción"" + 0.002*""piqué"" + 0.002*""rovegno"" + 0.002*""leer"" + 0.002*""candidato"" + 0.002*""año"" + 0.002*""boda"")","(1, 0.000*""vs"" + 0.000*""castillo"" + 0.000*""lima"" + 0.000*""perú"" + 0.000*""pedro"" + 0.000*""año"" + 0.000*""peruano"" + 0.000*""alianza"" + 0.000*""leer"" + 0.000*""piqué"")","(2, 0.007*""vs"" + 0.006*""castillo"" + 0.005*""pedro"" + 0.004*""lima"" + 0.004*""perú"" + 0.003*""peruano"" + 0.003*""leer"" + 0.003*""mundia